In [1]:
path_to_project="/home/martalo/Documentos/TFM/GPCR_variants"

import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install scipy
!{sys.executable} -m pip install bioservices
!{sys.executable} -m pip install biopython
!{sys.executable} -m pip install xlrd

from common_functions import *

import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist, squareform
import urllib

import mdtraj as md
import itertools
config(viewer='webgl')
from create_csv import * # this file is from Code/create_csv.py
import requests
import os
from matplotlib.ticker import FormatStrFormatter
from Bio.Data.IUPACData import protein_letters_3to1



/home/martalo/miniconda3/envs/my_new_env_name/lib/python3.6/site-packages/htmd/versionwarnings.py:29: UserWarning: As of HTMD 1.16 the default ACEMD version for all protocols has changed to version 3. If you want to use version 2 protocols change the _version argument in the protocols or add `config(acemdversion=2)` to the beginning of your scripts. To disable this warning run once: `from htmd import _disableWarnings; _disableWarnings('1.16');`
  , UserWarning)
/home/martalo/miniconda3/envs/my_new_env_name/lib/python3.6/site-packages/htmd/versionwarnings.py:33: UserWarning: As of HTMD 1.16 the default number of threads HTMD spawns for calculations is set to 1. You can enable parallelism at your own risk using `config(njobs=-2)` in the beginning of your scripts. To disable this warning run once: `from htmd import _disableWarnings; _disableWarnings('1.16');`
  , UserWarning)


ffevaluate module is in beta version

Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. https://dx.doi.org/10.1021/acs.jctc.6b00049

HTMD Documentation at: https://www.htmd.org/docs/latest/

New devel HTMD version (1.24.1 python[3.6,<3.7.0a0,3.7,<3.8.0a0]) is available. You are currently on (1.19).There are several methods to update:    - Create a new conda env. using `conda create -n htmd1.24.1 htmd=1.24.1 -c acellera -c psi4 -c conda-forge`    - Create a brand new conda installation and run `conda install htmd -c acellera -c psi4 -c conda-forge`    - Run: `conda update htmd -c acellera -c psi4 -c conda-forge` (NOT RECOMMENDED)



In [2]:
currentdir ="/home/martalo/Documentos/TFM/GPCR_variants/Code/potential_high_impact_variants/"
parentdir = "/home/martalo/Documentos/TFM/GPCR_variants/"
datapath="/home/martalo/Documentos/TFM/GPCR_variants/Data/studied_GPCR_vars"
resultspath="/home/martalo/Documentos/TFM/GPCR_variants/Results/studied_GPCR_vars"
remainingpath="/home/martalo/Documentos/TFM/GPCR_variants/Results/studied_GPCR_vars/remaining2"

## Functions 

In [3]:
def create_excel_from_raw(file_family_raw, file_family_csv):
    '''Read the txt file of the raw family information and build a ordered csv in the result 
    path'''
    # open the raw file 
    first_line=True
    with open(os.path.join(datapath,file_family_raw),"r") as infile:
        # build a csv file with the ordered info: family, name, short name, UniPort Id of each of the structures
        with open(os.path.join(resultspath,file_family_csv), 'w') as outfile:    
            mywriter = csv.writer(outfile,delimiter=';')
            for line in infile:
                if first_line:
                    mywriter.writerow(['Family','Name','Short','Uniprot ID'])
                    first_line=False
                if len(line) - len(line.lstrip())> 8 and not first_line:
                    el_li=line.strip().split('","')
                    fam=el_li[2]
                    short_nm=el_li[10]
                    long_nm=el_li[11]
                    uniprot=el_li[15]
                    mywriter.writerow([fam,long_nm,short_nm,uniprot])
    infile.close()
    outfile.close()
    return print('The file', file_family_csv,'has been succesfully created')

def search_struc_pdbs(original_csv, extended_csv): 
    '''With the information of the family, name, short name and Uniport ID, create
    another file adding the Uniprot entry name and the information of the structure
    from pdb (three colums, if there is active, intermediate and active structure)'''
    u = UniProt()# 'connection' with Uniprot services             
    with open(os.path.join(resultspath,original_csv),"r") as infile:
        with open(os.path.join(resultspath,extended_csv), 'w') as outfile:   
            mywriter = csv.writer(outfile,delimiter=';') 
            myreader = csv.reader(infile, delimiter=';')
            for row in myreader:
                if row[0]=="Family":
                    mywriter.writerow(row+["Uniprot entry","Struc"])
                    continue
                struccell=""
                entry=""
                uprot=row[3]
                print("\n\n",row[2])
                if uprot:
                    data=u.quick_search("id:%s" % uprot)
                    if data:
                        entry=data[uprot]['Entry name'].lower()
                        struc_res=obtain_struc_pdb(entry)
                        if struc_res:
                            struccell=struc_res
                            print(struccell)
                        else:
                            temp=requests.get('http://gpcrdb.org/services/structure/template/'+entry).json()
                            if temp:
                                temp_res=obtain_struc_pdb(temp)
                                if temp_res:
                                    struccell="[Model]: "+temp_res
                                    print(struccell)
                                else:
                                    print("-----No struc for template")
                            else:
                                print("-----No template")
                    else:
                        print("-----Uprot ID not found")
                else:
                    print("-----No uprot ID")
            
                mywriter.writerow(row+[entry,struccell])
    infile.close()
    outfile.close()
    return print('The file', extended_csv,'has been succesfully created')

def add_colum_names(csv_file_new, csv_file_old):
    '''Add colum names to the struc file and store it into new file'''
    with open(os.path.join(resultspath,csv_file_old),"r") as strucfile_old:
        with open(os.path.join(resultspath,csv_file_new),"w") as strucfile_new:
            myreader= csv.reader(strucfile_old, delimiter=';')
            mywriter= csv.writer(strucfile_new, delimiter=';') 
            for myrow in myreader:#iterate over the struc file rows
                if myrow[0]=="Familiy":
                    newrow=['Family', 'Name', 'Short', 'Uniprot ID','Uniprot entry','Struc','Struc1','Struc2']
                    print(newrow)
                    mywriter.writerow(newrow)
                else:
                    mywriter.writerow(myrow)
    strucfile_new.close()
    strucfile_old.close()
    return None

def create_csv_file(variants_excel, variants_csv):
    '''Tranform an Excel xlsx file into a csv file'''
    varFileAux=pd.read_excel(os.path.join(datapath,variants_excel))
    varFileAux.to_csv(os.path.join(datapath,variants_csv), sep=";", index=False)
    return None

def merge_struc_cols(csv_new, csv_old):
    '''Merged the possible three 'struc' pdb values, linked by '&' '''
    mynewrow=[]
    with open(os.path.join(resultspath,csv_old),"r") as file_old:
        with open(os.path.join(resultspath,csv_new),"w") as file_new:
            myreader= csv.reader(file_old, delimiter=';')
            mywriter= csv.writer(file_new, delimiter=';') 
            for myrow in myreader:#iterate over the file rows
                count=0
                if myrow[0]=="Family":
                    mywriter.writerow(myrow[:-2])
                    continue
                else: 
                    mynewrow=[]
                    mynewrow[0:5]=myrow[0:5]#first four the same
                    cols=''
                    for i in range (5,len(myrow)):
                        if myrow[i]!='':
                            cols+=str(myrow[i])+' & '
                    colscut=cols[:-2]
                    mynewrow.append(colscut)
                    print(mynewrow)
                    
                    mywriter.writerow(mynewrow)
                    mynewrow=[]
    file_new.close()
    file_old.close()
    return None

def get_gene(entry_name):
    '''Obtain the ENSEMBL id of the gene of the entry name'''
    uprot_map=uniprot_mapping('ACC+ID', 'ENSEMBL_ID', entry_name)#tab-delimited output of the mapping
    ens_id= uprot_map.decode().strip("\n").split("\t")[-1]
    return ens_id

def short_AA(AA_three_letters):
    '''Tranform 3 letter AA to one letter AA'''
    amin=AA_three_letters.upper()
    d = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
     'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
     'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
     'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}
    return d[amin]


In [9]:
# check if all the class A receptors have data
listAll=[]
myprotFile='myprot_list_ALL.csv'
with open(os.path.join(resultspath,myprotFile),"r") as myprotfile:   
    myprotread = csv.reader(myprotfile, delimiter=';') # csv with struc info (index)
    for myrow in myprotread:#iterate over the struc file rows
        if myrow[0]=="Familiy": # write first colum of the new csv              
            continue
        elif myrow[2]=='CALCR':
            break
        else:
            if myrow[3]!='':
                listAll.append(myrow[3])
                
setFile=set()
setData=set()
data=[]
myvarFile='PTM_IMPACT/vars_A_impact_ptm_role2.csv'
with open(os.path.join(resultspath,myvarFile),"r") as myprotfile:   
    myprotread = csv.reader(myprotfile, delimiter=';') # csv with struc info (index)
    for myrow in myprotread:#iterate over the struc file rows
        if myrow[0]=="Number": # write first colum of the new csv              
            continue
        else:
            setFile.add(myrow[4])
            data=myrow[2]+' '+myrow[3]+' '+myrow[4]
            setData.add(data)
listFile=list(setFile)
listData=list(setData)
print(len(setFile))



267


In [11]:
# build the file that has the names of the remaining gpcrs
# notInFile=[]
# for element in listAll:
#     if element not in listFile:
#         notInFile.append(element)
# print(len(notInFile))     # list of remaining GPCRs

myprotFileRemaining='remaining3/myprot_list_remaining.csv'
with open(os.path.join(resultspath,myprotFile),"r") as myprotfile:   
    myprotread = csv.reader(myprotfile, delimiter=';') # csv with struc info (index)
    with open(os.path.join(resultspath,myprotFileRemaining),"w") as myprotfileNew: 
        myprotwrite = csv.writer(myprotfileNew, delimiter=';') # 
        for myrow in myprotread:#iterate over the struc file rows
            if myrow[0]=="Familiy": # write first colum of the new csv 
                myprotwrite.writerow(myrow)
                continue
            else:
                if myrow[3] in notInFile:
                    mynewrow=myrow
                    data=myrow[1]+' '+myrow[2]+' '+myrow[3]
                    print(data)
                    myprotwrite.writerow(mynewrow)

19
adrenoceptor alpha 2B ADRA2B P18089
dopamine receptor D4 DRD4 P21917
histamine receptor H1 HRH1 P35367
neuropeptides B and W receptor 1 NPBWR1 P48145
neuropeptides B and W receptor 2 NPBWR2 P48146
neuropeptide Y receptor Y6 (pseudogene) NPY6R Q99463
relaxin/insulin like family peptide receptor 3 RXFP3 Q9NSD7
somatostatin receptor 3 SSTR3 P32745
leukotriene B4 receptor LTB4R Q15722
leukotriene B4 receptor 2 LTB4R2 Q9NPC1
adenosine A3 receptor ADORA3 P0DMS8
  P04001
G protein-coupled receptor 1 GPR1 P46091
G protein-coupled receptor 20 GPR20 Q99678
G protein-coupled receptor 22 GPR22 Q99680
G protein-coupled receptor 146 GPR146 Q96CH1
G protein-coupled receptor 148 GPR148 Q8TDV2
MAS1 proto-oncogene like, G protein-coupled receptor MAS1L P35410
trace amine associated receptor 3, pseudogene TAAR3P Q9P1P4


In [12]:
#######
# faltaban 19 receptores DE LA CLASE A QUE NO SE HABIAN CONSIDERADO. Además
# voy a rehacer los 6 últimos  a partir de 'purinergic receptor P2Y10' (INCLUÍDO)
# que se hicieron a mano. Todos ellos están en el fichero 'remaining2/myprot_list_remaining.csv'


#### 1.2. Add the structure information in a new csv
Build another csv file (/Results/remaining2) that stores the same info as before plus the  Uniprot entry name and three possible extra columns with the pdb structures names (active, intermediate, inactive):

In [4]:
# Only call if the previous csv have changed
search_struc_pdbs("remaining3/myprot_list_remaining.csv", 'remaining3/myprot_list_struc_remaining_old.csv')

# correct the column names
add_colum_names("remaining3/myprot_list_struc_remaining.csv", "remaining3/myprot_list_struc_remaining_old.csv")

# Put struc2 and struc3 in the same column linked by '&'
merge_struc_cols("remaining3/myprot_list_struc_remaining_merged.csv", "remaining3/myprot_list_struc_remaining.csv")



 Short
-----Uprot ID not found


 OPN2
4ZWJ (Active)


 OPN4
-----No struc for template


 ECPN
[Model]: 1GZM (Intermediate), 6OFJ (Intermediate)


 OPN5
-----No struc for template


 OPN1SW
[Model]: 4ZWJ (Active)


 OPN1MW
[Model]: 1GZM (Intermediate), 6OFJ (Intermediate)


 OPN1LW
[Model]: 1GZM (Intermediate), 6OFJ (Intermediate)


 OR1A1
[Model]: 6K41 (Active)


 OR1G1
[Model]: 6LFL (Inactive), 6LFO (Active)


 OR2T11
[Model]: 4Z35 (Inactive)
The file remaining3/myprot_list_struc_remaining_old.csv has been succesfully created
['Family', 'Name', 'Short', 'Uniprot ID', 'Uniprot entry', 'Struc', 'Struc1', 'Struc2']
['Sensory receptors Opsins ', 'Rhodopsin', 'OPN2', 'P08100', 'opsd_human', '4ZWJ (Active) ']
['Sensory receptors Opsins ', 'Melanopsin', 'OPN4', 'Q9UHM6', 'opn4_human', '']
['Sensory receptors Opsins ', 'Opsin-3', 'ECPN', 'Q9H1Y3', 'opn3_human', '[Model]: 1GZM (Intermediate), 6OFJ (Intermediate) ']
['Sensory receptors Opsins ', 'Opsin-5', 'OPN5', 'Q6U736', 'opn5_human', ''

#### 1.3. Build the file 'variants_ALL_GNOMAD.csv' 
It will contain the GPCR pdb structure info plus the info of the variants that comes from GnomAD. We will use gnomAD v2 data set which contains data from 125,748 exomes and 15,708 whole genomes from unrelated individuals sequenced as part of various disease-specific and population genetic studies, totalling 141,456 individuals, and is aligned against the GRCh37 reference sequence (the latest version v3 has less exomes information, recommended if you are interesting in non-coding regions, not our case).
https://gnomad.broadinstitute.org/

First, we have an example on how to make queries in GnomAD, as they do not have an API:

In [5]:
########################################################
# ESTE CODIGO HA SIDO EDITADO (en comparación con el que se usó para role2)
# Se ha tenido que añadir el reference genome: GRCh37
########################################################


# Query GnomAD to get: hgvsp (to get segment position, original AA and variant AA), allele freq,
#num, count and annotation from an Ensembl ID
import pprint
prettyprint = pprint.PrettyPrinter(indent=2).pprint

def fetch(jsondata, url="https://gnomad.broadinstitute.org/api"):
    # The server gives a generic error message if the content type isn't
    # explicitly set
    headers = {"Content-Type": "application/json"}
    response = requests.post(url, json=jsondata, headers=headers)
    json = response.json()
    if "errors" in json:
        raise Exception(str(json["errors"]))
    return json

def get_variant_list(gene_id, dataset="gnomad_r2_1"):
    # Note that this is GraphQL, not JSON.
    # Possible data to retrieve:
    # pos	rsid	ref	alt	consequence	exome	flags	lof	consequence_in_canonical_transcript
    #gene_symbol	hgvsc	lof_filter	lof_flags	hgvsp	reference_genome	variant_id	
    #genome_af	genome_ac	genome_an	genome_ac_hemi	genome_ac_hom	genome	exome_af	
    #exome_ac	exome_an	exome_ac_hemi	exome_ac_hom
    
    # We need the hgvsp (segNum, originalAA and variantAA), allele freq, allele num, allele 
    # count, consequence (get only MVs). We lack number of homozygotes genome_ac_hom
    fmt_graphql = """
    {
        gene(gene_id: "%s", reference_genome: GRCh37) {
          variants(dataset: %s) {
            hgvsp            
            genome {
                af
                ac
                an
            }
            consequence
            rsid
          }
        }
      }
    """# rsid, variant_id: variantId -> maybe useful
    
    # This part will be JSON encoded, but with the GraphQL part left as a
    # glob of text.
    req_variantlist = {
        "query": fmt_graphql % (gene_id, dataset),
        "variables": {}
        }
    response = fetch(req_variantlist)#json format
    return response["data"]["gene"]["variants"]
    #return response

# Examples of how the info is obtained, the results can be compared with the ones from
# gnomad_python_api.git in the output folder, file: variants.tsv
#prettyprint(get_variant_list("ENSG00000169174"))
# print(get_variant_list("ENSG00000135569"))
        

In [8]:
# retrieve MV data of the GPCR of the index file

def find_variants_gnomad(index_csv, final_csv):
    '''Use the info of the index file to look for the variants stored in GnomAD and store
    them in the final_csv'''
    
    non_identified=0 # keep track of the non-identified GPCRs
    not_entry=0
    with open(os.path.join(resultspath,index_csv),"r") as myprotfile:
    # Build new file merging structure info (index file) + variants info (GnomAD), containing:
    # Family	Name	Short	Uniprot ID	Uniprot entry	Struc	Ligandtype	gene
    #GPCRdb	SequenceNumber	Segment	GPCRdbWT	NMaa	MutationType	Allele Count
    #Allele Frequency	Allele Number	Number of Homozygotes	sift_word	sift_score
    #polyphen_word	polyphen_score	GProteinInteraction	ArrestinInteraction	
    #ActivationPathway	MicroSwitch	SodiumPocket	foldchangeMaxAbs	diseaseId	
    #score	diseaseName	Type	PTMsite	LB_structure	LB_fam

        with open(os.path.join(resultspath,final_csv), 'w') as outfile:   
            myprotread = csv.reader(myprotfile, delimiter=';') # csv with struc info (index)
            mywriter = csv.writer(outfile,delimiter=';') # final csv
            for myrow in myprotread:#iterate over the struc file rows
                if myrow[0]=="Family": # write first colum of the new csv
                    newrow=myrow+['Ligandtype','gene','varID', 'GPCRdb', 'SequenceNumber', 'Segment', 'GPCRdbWT', 'NMaa', 'MutationType', 'Allele Count', 'Allele Frequency', 'Allele Number', 'Number of Homozygotes', 'sift_word', 'sift_score', 'polyphen_word', 'polyphen_score', 'GProteinInteraction', 'ArrestinInteraction', 'ActivationPathway', 'MicroSwitch', 'SodiumPocket', 'foldchangeMaxAbs', 'diseaseId', 'score', 'diseaseName', 'Type', 'PTMsite', 'LB_structure', 'LB_fam']
                    mywriter.writerow(newrow)
                    # Containing the elements of struc file plus the interesting ones of the var 
                    continue
                entry=myrow[4] # colum 4 from csv with struc info: Uniprot entry
#                 print('Entry',entry)
                if not entry: # if there is not Uniprot entry
                    mywriter.writerow(myrow)
                    not_entry+=1
                    continue
                
                ens_id= myrow[6] # get Ensembl ID
                print('Analizing:',entry,ens_id)
                no_id=False
#                 try: # now check this Ensembl ID in gnomAD
                gnom=get_variant_list(ens_id)  # list of dict of the variants found in
#                 #gnomAD of the entry
#                 except:
#                     print('No GnomAD entry with this ensG')
#                     no_id=True
#                     non_identified+=1
                
                # if the search in GnomAD has been succesfull
                if not no_id:
                    print('Esemble ID in GnomAD')
                    # get the structure data of the protein from GPCRdb
                    gpcr_pos_info=get_gpcr_pos_info(entry)
                    # keys as the 'sequence_number' (position) and values -> protein_segment, gnum

                    found=False                    
                    for gnom_var in gnom:#iterate over the found variants for the entry
                        if gnom_var["consequence"] =="missense_variant":
#                             print('MV in GnomAD')
                            found=True
                            
                            var_id=gnom_var['rsid']
                            
                            # get the seqNum, originalAA and variantAA from 'hgvsp' 
                            AAdata=gnom_var['hgvsp']
                            fromAA_long=AAdata[2:5] # Ex: Ala
                            toAA_long=AAdata[-3:] # Ex: Gly
                            fromAA=short_AA(fromAA_long) # Ex: A
                            toAA=short_AA(toAA_long) # Ex: G
                            seqNum=int(re.findall('\d+', gnom_var['hgvsp'] )[0])# get only the numbers
#                             print('seqNum:', seqNum)
                            
                            # get allele data
                            if gnom_var["genome"]:
                                gen_dict=gnom_var["genome"]
                                allele_count=gen_dict['ac']
                                allele_freq=gen_dict['af']
                                allele_num=gen_dict['an']
                            else:
                                allele_count='?'
                                allele_freq='?'
                                allele_num='?'
                            
                            #look for the gnum of the variant position
                            gpcrNum=""
                            try: 
                                this_pos_info=gpcr_pos_info[seqNum]
                                if this_pos_info["display_generic_number"]:
                                    gpcrNum=this_pos_info["display_generic_number"]
                                Segment=this_pos_info["protein_segment"]
                            except:
                                print('Position/SeqNum not found')                                
                                gpcrNum='?'
                                Segment='?'
                                
                            hom_count='?'###### Lack from GnomAD
                            Ligandtype="?"
                            MutationType=check_aa_type_changed(fromAA,toAA)
                            sift_word="?"
                            sift_score="?"
                            polyphen_word="?"
                            polyphen_score="?"
                            GProteinInteraction="?"
                            ArrestinInteraction="?"
                            ActivationPathway="?"
                            MicroSwitch="?"
                            SodiumPocket="?"
                            foldchangeMaxAbs="?"
                            diseaseId="?"
                            score="?"
                            diseaseName="?"
                            Type="?"
                            PTMsite="?"
                            LB_structure="?"
                            LB_fam="?"
                            
                            mywriter.writerow(myrow + [Ligandtype, ens_id, var_id, gpcrNum, seqNum, Segment, fromAA, toAA, MutationType, allele_count, allele_freq, allele_num, hom_count, sift_word, sift_score, polyphen_word, polyphen_score, GProteinInteraction, ArrestinInteraction, ActivationPathway, MicroSwitch, SodiumPocket, foldchangeMaxAbs, diseaseId, score, diseaseName, Type, PTMsite, LB_structure, LB_fam])
                    if not found:
                        print("No variants found for %s" % entry)
                        mywriter.writerow(myrow)
                else:#no result in GNOMAd
                    print("No results in GNOMAD for the EnsemblID of",entry, ens_id)
                    var_id=''
                    fromAA='?'
                    toAA='?'
                    allele_count='?'
                    allele_freq='?'
                    allele_num='?'
                    Segment='?'
                    seqNum='?'
                    gpcrNum='?'
                    hom_count='?'###### Lack from GnomAD
                    Ligandtype="?"
                    MutationType='?'
                    sift_word="?"
                    sift_score="?"
                    polyphen_word="?"
                    polyphen_score="?"
                    GProteinInteraction="?"
                    ArrestinInteraction="?"
                    ActivationPathway="?"
                    MicroSwitch="?"
                    SodiumPocket="?"
                    foldchangeMaxAbs="?"
                    diseaseId="?"
                    score="?"
                    diseaseName="?"
                    Type="?"
                    PTMsite="?"
                    LB_structure="?"
                    LB_fam="?"
                    mywriter.writerow(myrow + [Ligandtype, ens_id, var_id, gpcrNum, seqNum, Segment, fromAA, toAA, MutationType, allele_count, allele_freq, allele_num, hom_count, sift_word, sift_score, polyphen_word, polyphen_score, GProteinInteraction, ArrestinInteraction, ActivationPathway, MicroSwitch, SodiumPocket, foldchangeMaxAbs, diseaseId, score, diseaseName, Type, PTMsite, LB_structure, LB_fam])
    print('Without ensembl ID:', non_identified)
    print('Without entry:', not_entry)
    return None

find_variants_gnomad("remaining3/myprot_list_struc_remaining_merged_ENSG.csv","remaining3/GNOMAD/variants_A_remaining_GNOMAD.csv") 
# Then the first column 'gene' has ben manually erased because it was duplicated

Analizing: opsd_human ENSG00000163914
Esemble ID in GnomAD
Analizing: opn4_human ENSG00000122375
Esemble ID in GnomAD
Position/SeqNum not found
Position/SeqNum not found
Position/SeqNum not found
Position/SeqNum not found
Position/SeqNum not found
Position/SeqNum not found
Position/SeqNum not found
Position/SeqNum not found
Position/SeqNum not found
Position/SeqNum not found
Position/SeqNum not found
Position/SeqNum not found
Position/SeqNum not found
Position/SeqNum not found
Position/SeqNum not found
Position/SeqNum not found
Position/SeqNum not found
Position/SeqNum not found
Position/SeqNum not found
Position/SeqNum not found
Position/SeqNum not found
Position/SeqNum not found
Position/SeqNum not found
Analizing: opn3_human ENSG00000054277
Esemble ID in GnomAD
Analizing: opn5_human ENSG00000124818
Esemble ID in GnomAD
Position/SeqNum not found
Position/SeqNum not found
Position/SeqNum not found
Position/SeqNum not found
Position/SeqNum not found
Position/SeqNum not found
Position/S

In [41]:
# El anterior fichero contiene las variants de las 24 GPCRs de la clase A que faltaban en el
# fichero final 'role2.csv'. Había un problema parece con el ENSID, no concuerda el de GnomAD
# con el de Uniprot. Para solventarlo se han cogido a mano sus ENSG de GnomAD (introduciendo los
# 24 nombres 'Short') y se has repetido las queries.
# Nota: habia un par que no mostraban variants. 

# Revisar:
# 1. Por qué las diferencias entre ENSG en Uniprot y GnomAD?
# 2. Qué supone utilizar el GRCh37 o GRCh38? Cual se utilizaba antes, cuando hice las quieries
# del resto, que no se especificaba el refGenome? Son el resto de ENSG concordantes?

#### 1.4 Retrieve information from DisGeNET
Now, we want to get the information of DisGeNEt to fill in the information of columns: 'diseaseId', 'score', 'diseaseName' of the previous file. To do so, we will use the rest API disgenet.org/api/, specially, the Variant-Disease Associations (VDAs). The VDAs service provides an interface for accessing variant-disease associations from DisGeNET. The VDAs can be retrieved for a specific variant (ex: rs121909213), or list of variants. It is recommended that the list of query variants is not longer than 100 entities. The service returns the results formated in TSV, JSON, or XML.

In [9]:
def request_disgenet(varID):
    '''Get variant relation with a disease from DisgeNet using the variant id (Ex:rs121909211)'''
    # https://www.disgenet.org/api/#/VDA/vdaByVariant
    base = 'https://www.disgenet.org/api'
    tool = 'vda/variant'
    url = base+'/'+tool+'/'+varID
    response =  urllib.request.urlopen(url)
    return (response.read())

def get_disease_info(req_response):
    '''Select only the three important values of the DisgeNET response for each variant,
    Id, disease name and score'''
    # Available information:
# {"variantid":"rs121909211","gene_symbol":"TGFBI","variant_dsi":"0.724","variant_dpi":"0.2",
#  "variant_consequence_type":"missense variant","diseaseid":"C1275685",
#  "disease_name":"Avellino corneal dystrophy","disease_class":"C16;C11",
#  "disease_class_name":"   Congenital, Hereditary, and Neonatal Diseases and Abnormalities;    
#  Eye Diseases","disease_type":"disease","disease_semantic_type":"Disease or Syndrome",
#  "score":0.9,"ei":1.0,"year_initial":1998,"year_final":2019,"source":"ALL"}

    req_list = json.loads(req_response)
    list_diseases=[]
    dis_dic={}
    print('Elements of the returned list of related diseases:')
    for element in req_list:
        dis_dic['disease_id']=element['diseaseid']
        dis_dic['score']=element['score']
        dis_dic['diseaseName']=element['disease_name']
        list_diseases.append(dis_dic)
        print(dis_dic)
        dis_dic={}
    return list_diseases


def major_disease(list_of_diseases):
    '''Returns a tupple with the dictionary of the disease with the higest score and 
    the total number of related diseases'''
    mvp=list_of_diseases[0] # most important disease
    number_dis=len(list_of_diseases) # number of related diseases
    for element in list_of_diseases:
        if element['score'] > mvp['score'] :
            mvp=element
    print('The disease with higher score is:',mvp)
    print('Number of related diseases:',number_dis)
    return(mvp, number_dis)

# Examples:
# related_diseases= get_disease_info(request_disgenet('rs121909211')) #list of diseases
# most_imp_dis=major_disease(related_diseases)
# print(most_imp_dis[1])

In [14]:
#disCols=[29,30,31]  'diseaseId'-> 'diseaseid', 'score' -> 'score' (Score VDA: Variant-Disease 
# Asociation Score),'diseaseName' -> 'disease_name'

def add_dis_info(var_filtered, var_disgenet):
    '''Add the information of DisGeNET, id, socre, name, and the number of related
    diseases for each varID'''
    with open(os.path.join(resultspath,var_filtered),"r") as myprotfile:
        with open(os.path.join(resultspath,var_disgenet), 'w') as outfile:   
            myprotread = csv.reader(myprotfile, delimiter=';') # csv with struc info (index)
            mywriter = csv.writer(outfile,delimiter=';') # final csv
            for myrow in myprotread:#iterate over the struc file rows
                if myrow[0]=="Family": # write first colum of the new csv
                    mynewrow=myrow
                    mynewrow.append("related_diseases")
                    mywriter.writerow(mynewrow)                
                    continue
                else:
                    varID=myrow[8]
                    if varID=='':
                        continue
                    try:
                        related_dis= get_disease_info(request_disgenet(varID)) #list of diseases
                    except:
#                         print('No matches in DisGeNET for:', varID)
                        mywriter.writerow(myrow)
                        continue
                    imp_info=major_disease(related_dis)
                    most_imp_dis=imp_info[0]# disease info with the highest score
                    num_dis=imp_info[1] # number of related diseases
                    mynewrow=myrow
                    mynewrow[29]=most_imp_dis['disease_id']
                    mynewrow[30]=most_imp_dis['score']
                    mynewrow[31]=most_imp_dis['diseaseName']
                    mynewrow.append(num_dis)
                mywriter.writerow(mynewrow) 
    myprotfile.close()
    outfile.close()
    return (print('The file', var_disgenet,'has been succesfully created'))
# fist delete the added column of 'Gene' from 'variants_A_remaining_GNOMAD.csv'             
add_dis_info("remaining3/GNOMAD/variants_A_remaining_GNOMAD.csv", "remaining3/DISGENET/variants_A_remaining_GNOMAD_DISGENET.csv")

Elements of the returned list of related diseases:
{'disease_id': 'C3151001', 'score': 0.8, 'diseaseName': 'Retinitis Pigmentosa 4'}
{'disease_id': 'C0035334', 'score': 0.75, 'diseaseName': 'Retinitis Pigmentosa'}
{'disease_id': 'C0339525', 'score': 0.02, 'diseaseName': 'Autosomal dominant retinitis pigmentosa'}
{'disease_id': 'C0152200', 'score': 0.01, 'diseaseName': 'Achromatopsia'}
{'disease_id': 'C0302129', 'score': 0.01, 'diseaseName': 'Achromatopsia 1'}
{'disease_id': 'C1562230', 'score': 0.01, 'diseaseName': 'Retinochoroidopathy'}
The disease with higher score is: {'disease_id': 'C3151001', 'score': 0.8, 'diseaseName': 'Retinitis Pigmentosa 4'}
Number of related diseases: 6
Elements of the returned list of related diseases:
{'disease_id': 'C0035334', 'score': 0.71, 'diseaseName': 'Retinitis Pigmentosa'}
{'disease_id': 'C3151001', 'score': 0.7, 'diseaseName': 'Retinitis Pigmentosa 4'}
The disease with higher score is: {'disease_id': 'C0035334', 'score': 0.71, 'diseaseName': 'Reti

Elements of the returned list of related diseases:
{'disease_id': 'C0339537', 'score': 0.7, 'diseaseName': 'Cone monochromatism'}
The disease with higher score is: {'disease_id': 'C0339537', 'score': 0.7, 'diseaseName': 'Cone monochromatism'}
Number of related diseases: 1
The file remaining3/DISGENET/variants_A_remaining_GNOMAD_DISGENET.csv has been succesfully created


#### 1.4 Compute the Impact Scores: SIFT, PolyPhen

In [12]:

def request_VEP(varID):
    '''Get variant SIFT and PolyPhen2 from VEP (Ensembl) using the variant id (Ex:rs121909211)'''
#     server = "https://rest.ensembl.org" #GRCh38
    server="https://grch37.rest.ensembl.org" #GRCh37
    ext = "/vep/human/id/"+varID+"?"     
    
    r = requests.get(server+ext, headers={ "Content-Type" : "application/json"}) 

    if not r.ok:
        r.raise_for_status()
        sys.exit()

    decoded = r.json()
#     print(decoded)
    return(decoded)

# Examples:
# response=request_VEP("rs553609439")
# response=request_VEP("rs199805893")
# response=request_VEP("rs759659059")

def filter_VEP_response(VEPresponse,ensID,varID):
    '''Filer the respose, list type, from VEP, after quering a varID. Get the 
    'sift_prediction','sift_score','polyphen_prediction','polyphen_score' for 
    each transcriptID (after checking is the correct EnsemblID)'''
    pred_dic={}
    keys=['sift_prediction','sift_score','polyphen_prediction','polyphen_score']
    for element in VEPresponse:
        prediction_list=element['transcript_consequences']
#         pred_dic={}
        for dictionary in prediction_list:
            predictions=True
#             print(dictionary)
#             print(dictionary['gene_id'], ensID)
            if dictionary['gene_id']==ensID:# check ensemblID
                print('ensID ok')
                transID=dictionary['transcript_id'] # store each transcriptID
#                 print('transID')
                for value in keys:# check all the predictions exist
                    if value not in dictionary.keys():
                        predictions=False  
#                         print('Not all the keys in the dictionary')
                    if predictions:
                        pred_dic[transID]={'Sift': [ dictionary['sift_score'], dictionary['sift_prediction'] ],
                                            'Polyphen':[dictionary['polyphen_score'], dictionary['polyphen_prediction'] ]}
            else:
                print('ensid not ok')
    # returned dictionary with keys as transcripID and values as the prediction values 
    print(pred_dic)
    return pred_dic

# print(filter_VEP_response(response, "ENSG00000222040","rs553609439"))
# print(filter_VEP_response(response, "ENSG00000185149","rs199805893"))


def add_sift_polyphen(old,new):
    '''Retrieve the information from VEP using the varID and EnsID to fill in the score 
    impact columns for SIFT and PolyPhen. As there could be more than one transcript
    for the same geneID and varID store all the values separed by white spaces'''
    with open(os.path.join(resultspath,old),"r") as myprotfile:
        with open(os.path.join(resultspath,new), 'w') as outfile:   
            myprotread = csv.reader(myprotfile, delimiter=';') 
            mywriter = csv.writer(outfile,delimiter=';') # final csv
            for myrow in myprotread:
                if myrow[0]=="Family": # write first colum of the new csv
                    mynewrow=myrow
                    mywriter.writerow(myrow)                
                    continue
                elif myrow[8]!='':
                    mynewrow=myrow
                    #check ensemblID and varID  to get the info
                    # keys=['sift_prediction','sift_score','polyphen_prediction','polyphen_score']
                    ensID=myrow[7]
                    varID=myrow[8]
                    print(varID)
                    VEPresp=request_VEP(varID)#list (long info from VEP)
                    filteredVEPresp=filter_VEP_response(VEPresp,ensID,varID)# dictionary 
                    #19,20,21,22
#                     print(varID,filteredVEPresp)
                    
                    if len(filteredVEPresp.keys())==1:
                        for transID in filteredVEPresp:#only one
                            auxDic=filteredVEPresp[transID]
                            siftData=auxDic['Sift']
                            mynewrow[20]=(transID+':'+str(siftData[0]))
                            mynewrow[19]=(transID+':'+siftData[1])
                            ppData=auxDic['Polyphen']
                            mynewrow[22]=(transID+':'+str(ppData[0]))
                            mynewrow[21]=(transID+':'+ppData[1])
                    elif len(filteredVEPresp.keys())>1:# more than one transcript
                        for transID in filteredVEPresp:
                            auxDic=filteredVEPresp[transID]
                            siftData=auxDic['Sift']
                            ppData=auxDic['Polyphen']
                            if mynewrow[19]=='?' and mynewrow[20] == '?':
                                mynewrow[20]=(transID+':'+str(siftData[0])+'   ')
                                mynewrow[19]=(transID+':'+siftData[1]+'   ')
                                mynewrow[22]=(transID+':'+str(ppData[0])+'   ')
                                mynewrow[21]=(transID+':'+ppData[1]+'   ')
                            else:
                                mynewrow[20]+=(transID+':'+str(siftData[0])+'   ')
                                mynewrow[19]+=(transID+':'+siftData[1]+'   ')
                                mynewrow[22]+=(transID+':'+str(ppData[0])+'   ')
                                mynewrow[21]+=(transID+':'+ppData[1]+'   ')
                    mywriter.writerow(mynewrow)
                    
        return print('The file', new, 'has been succesfully created.')

# add_sift_polyphen("remaining3/DISGENET/variants_A_remaining_GNOMAD_DISGENET.csv", "remaining3/PRED/variants_A_remaining_GNOMAD_DISGENET_PRED.csv")                    

rs782122931
ensID ok
ensID ok
ensID ok
ensid not ok
ensid not ok
ensid not ok
{'ENST00000369935': {'Sift': [0, 'deleterious'], 'Polyphen': [0.921, 'probably_damaging']}}
rs781899063
ensID ok
ensID ok
ensID ok
ensid not ok
ensid not ok
ensid not ok
{'ENST00000369935': {'Sift': [0.02, 'deleterious'], 'Polyphen': [0.933, 'probably_damaging']}}
rs781807082
ensID ok
ensID ok
ensID ok
ensid not ok
ensid not ok
ensid not ok
{'ENST00000369935': {'Sift': [0.22, 'tolerated'], 'Polyphen': [0.162, 'benign']}}
rs782614830
ensID ok
ensID ok
ensID ok
ensid not ok
ensid not ok
ensid not ok
{'ENST00000369935': {'Sift': [0.93, 'tolerated'], 'Polyphen': [0.006, 'benign']}}
rs200470120
ensID ok
ensID ok
ensID ok
ensid not ok
ensid not ok
ensid not ok
{'ENST00000369935': {'Sift': [0.07, 'tolerated'], 'Polyphen': [0.081, 'benign']}}
rs782658018
ensID ok
ensID ok
ensID ok
ensid not ok
ensid not ok
ensid not ok
{'ENST00000369935': {'Sift': [0.06, 'tolerated'], 'Polyphen': [0.524, 'possibly_damaging']}}
rs2015

{'ENST00000369935': {'Sift': [0.02, 'deleterious'], 'Polyphen': [0.991, 'probably_damaging']}, 'ENST00000430054': {'Sift': [0, 'deleterious_low_confidence'], 'Polyphen': [0.996, 'probably_damaging']}}
The file remaining3/PRED/variants_A_remaining_GNOMAD_DISGENET_PRED_last.csv has been succesfully created.


#### 1.6 Rearrange variant information

#### 1.6.1 Functional impact scores merging and  and relation with diseases
Change the way in which we see the impacts. Set '0' if both predictors are below the scores that set the variants as 'deleterious' (SIFT) and 'probably_damaging' (PolyPhen2). Set '1' if just one predictor catalogs the variant as damaging and '2' if both predictors do.

*Note: the PolyPhen2 tag, 'possibly damaging' is not considered high impact score (approx 0.7).*


In [13]:
def split_and_add(listPredictions, dicPredictions, position):
    '''split the strings that come from the csv, colums of the predictions, by ':' and
    them to a dictionary. Used by the function 'fuse_preds' '''
    for element in listPredictions:
        if element != '':
            items=element.split(':')#[transID, predictionValue]
            if position==0:#void dictionary
                dicPredictions[items[0]]=[items[1]]
            else:    
                dicPredictions[items[0]].append(items[1])
    return None

def fuse_preds(siftScore,siftPred,ppScore,ppPred):
    '''Build a dictionary with keys as transID and values as siftScore, siftPred,
    ppScore, ppPred'''
    returnDic={}
    #sift
    predListSift=siftPred.split('   ')# transID1:'deleterious'1, transID2:'deleterious'2,...
    scoreListSift=siftScore.split('   ')# transID1:score1,...
    #poplyphen
    predListPP=ppPred.split('   ')# transID1:'deleterious'1, transID2:'deleterious'2,...
    scoreListPP=ppScore.split('   ')# transID1:score1,...
    lists=[predListSift,scoreListSift,predListPP,scoreListPP]
    index=-1
    for element in lists:
        index+=1
        split_and_add(element,returnDic,index)#split by ':'
    return returnDic


# examples:
# sp="ENST00000216629:tolerated   ENST00000553356:deleterious   ENST00000611804:tolerated   " 
# ss="ENST00000216629:0.06   ENST00000553356:0.04   ENST00000611804:0.06 "
# ppp="ENST00000216629:probably_damaging   ENST00000553356:probably_damaging   ENST00000611804:probably_damaging"
# pps="ENST00000216629:0.983   ENST00000553356:0.972   ENST00000611804:0.983   "

# sp="ENST00000555147:deleterious   ENST00000555427:deleterious   ENST00000639847:deleterious   "
# ss="ENST00000555147:0.03   ENST00000555427:0.03   ENST00000639847:0.03   "
# ppp="ENST00000555147:benign   ENST00000555427:benign   ENST00000639847:benign   "
# pps="ENST00000555147:0.248   ENST00000555427:0.342   ENST00000639847:0.248   "
# dicProva=fuse_preds(sp,ss,ppp,pps)

def get_trans_uniprot(entry_name):
    '''Obtain the ENSEMBL transIDs list of the gene with the protein entry name.
    Used by the function 'clean_trans' '''
    uprot_map=uniprot_mapping('ACC+ID', 'ENSEMBL_TRS_ID', entry_name)#tab-delimited output of the mapping
    lines=uprot_map.decode().split("\n")
    tIDs=[]
    for elements in lines:
        if elements!='' and elements !='From\tTo':
            tIDs.append(elements.split("\t")[-1])
    return tIDs
#example
# print(get_trans_uniprot('Q13639'))

def get_trans(gene_name):
    '''Obtain the transcript ids of GRCh37 from Ensebl'''
    server = "https://grch37.rest.ensembl.org"
    ext = "/xrefs/symbol/homo_sapiens/"+gene_name+"?object_type=transcript"

    r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})

    if not r.ok:
        r.raise_for_status()
        sys.exit()

    decoded = r.json()
    print(repr(decoded))
    transList=[]
    for element in decoded:#list with dictionaries
        value=element['id']
        if value[0:4]=='ENST':# 
            transList.append(value)
    return transList
 #example
# print(get_trans('MAS1L'))


First we build a dictionary, keys as all the Uniprot names from the 'index' file. As value we will have a list with the corresponding transcript ID of each entry name from Uniprot: 

In [14]:
def read_prots(fileProt):
    '''Build the dictionary of receptors from the index file, keys as prot names (short name), 
    values (list) the transIDs.'''
    listProts=[]
    count=0
    with open(os.path.join(resultspath,fileProt),"r") as myvarfile:
        myvarread = csv.reader(myvarfile, delimiter=';') 
        for myrow in myvarread:
            if myrow[0]=="Family": #first row
                continue            
            else:
                listProts.append(myrow[2])
    myvarfile.close()
#     print(listProts)
    return listProts

protList=read_prots(os.path.join(resultspath,'remaining3/myprot_list_struc_remaining.csv'))

transProtDic={}# key: shortName, value:[transIDs]
for element in protList:
    transProtDic[element]=get_trans(element)

[{'id': 'ENST00000296271', 'type': 'transcript'}]
[{'id': 'ENST00000372071', 'type': 'transcript'}]
[{'type': 'transcript', 'id': 'ENST00000366554'}]
[{'id': 'ENST00000371211', 'type': 'transcript'}]
[{'id': 'ENST00000249389', 'type': 'transcript'}]
[{'type': 'transcript', 'id': 'ENST00000369935'}, {'type': 'transcript', 'id': 'ENST00000595290'}]
[{'id': 'ENST00000369951', 'type': 'transcript'}, {'type': 'transcript', 'id': 'ENST00000597195'}]
[{'id': 'ENST00000304094', 'type': 'transcript'}]
[{'id': 'ENST00000328890', 'type': 'transcript'}]
[{'id': 'ENST00000330803', 'type': 'transcript'}]


In [15]:
transProtDic # as I dont understand wht this results are not the same from VEP I will 
# just consider the transcript that coincides

{'OPN2': ['ENST00000296271'],
 'OPN4': ['ENST00000372071'],
 'ECPN': ['ENST00000366554'],
 'OPN5': ['ENST00000371211'],
 'OPN1SW': ['ENST00000249389'],
 'OPN1MW': ['ENST00000369935', 'ENST00000595290'],
 'OPN1LW': ['ENST00000369951', 'ENST00000597195'],
 'OR1A1': ['ENST00000304094'],
 'OR1G1': ['ENST00000328890'],
 'OR2T11': ['ENST00000330803']}

Now, code to generate the file with the imapct (0,1,2) and PTM values:

In [16]:
def clean_trans(transPredDic, entryName,dictOfTrans=transProtDic):
    '''From all the trancript values from VEP only keep the transcript of the correct protein,
    filter the dicttionary, return the correct one.'''
    correctTrans=dictOfTrans[entryName]# list of the correct transcripts of the protein
#     print('Correnct transID:',correctTrans)
    correctDic={}
#     print('Intro dic:',transPredDic)
    for key, value in transPredDic.items():
        if key in correctTrans:
            correctDic[key]=value
    print(correctDic)
    return correctDic

def write_newvar_line(orgRow,impactPred):
    '''New structure of the rows '''
    newrow=[]
    for i in range(0,26):
        newrow.append('')
#     for i in range(0,6):
#         newrow[i]=orgRow[i]
#     for i in range(6,17):
#         newrow[i]=orgRow[i+1]
    newrow[0:6]=orgRow[0:6]
    newrow[6:17]=orgRow[7:18]
    
    #we don't know the role yet    
    newrow[17]=''# we don't know the role of the vars in all GPCRs, the gnum varies between classes
    #####################################
    newrow[18]=impactPred#impact prediction:0, 1 or 2
    for i in range (19,22):#diseaseID,score, name
        newrow[i]=orgRow[i+10]
    newrow[22]=orgRow[-1]#numer of diseases
    
    newrow[23]='?'#foldchange
        
    return newrow

def compute_impact(dictionary):
    '''Check if the variant is predicted to have impact by one or two prediction tools, 
    if different, take the max'''
    impactos=[0]
    for trasid in dictionary:
        listPrediction=dictionary[trasid]
        if listPrediction[0]=='deleterious' and listPrediction[2]=='probably_damaging':
            impactos.append(2)#both predict impact
        elif listPrediction[0]=='deleterious' or listPrediction[2]=='probably_damaging':
            impactos.append(1)
    return max(impactos)
#Example
# aux={'ENST00000555147': ['benign','0.03',  'benign','0.248',], 'ENST00000639847': [ 'benign','0.03',  'benign','0.248']}
# compute_impact(aux)


In [17]:
columNamesORG=["Family0","Name0","Short1","Uniprot ID2","Uniprot entry3","Struc4","Ligandtype5",
               "gene6","varID7","GPCRdb8","SequenceNumber9","Segment10","GPCRdbWT11","NMaa12",
               "MutationType13","Allele Count14","Allele Frequency15","Allele Number16",
               "Number of Homozygotes17","sift_word18","sift_score19","polyphen_word20",
               "polyphen_score21","GProteinInteraction22","ArrestinInteraction23",
               "ActivationPathway24","MicroSwitch25","SodiumPocket26","foldchangeMaxAbs27",
               "diseaseId28","score29","diseaseName30","Type31","PTMsite32",
               "LB_structure33","LB_fam34","related_diseases35"]

# OUT:
# "ligandType" = row 6
# "Number of Homozygotes" = row 18

# "activationPathway"=row 24
# is out because it is descrived as 'Class A activation pathway positions: '3x46','6x37','7x53' 
# and none of those positions are considered'

# "LB_fam" = row 35 (ligand binding interaction observed in a crystal structure within 
#            the same receptor family)

# REPLACED:
# "sift_word","sift_score","polyphen_word","polyphen_score"=row 19,20,21,22
# -> to 'Impact prediction', 0,1,2 (none, one, both predict impact)

# "GProteinInteraction","ArrestinInteraction","MicroSwitch","SodiumPocket","LB_structure"
# = row22,23,25,26,34
# those will be changed by 'Role': BS, IBS/ionicLock (CM),IBS(arrestin/gprotint/chainA/B)

# DOUBTS:
# "foldchangeMaxAbs"= maximum (absolute) observed foldchange in in vitro pharmacological experiments
# "PTMsite" = PTM site (yes/no) 
# "Type" = PTM type (replaced by PTMtype)


columNamesNEW=["Family","Name","Short","Uniprot ID","Uniprot entry","Struc","gene","varID",
               "GPCRdb","SequenceNumber","Segment","GPCRdbWT","NMaa","MutationType",
               "Allele Count", "Allele Frequency","Allele Number","Role","Impact Prediction",
               "diseaseId","score","diseaseName","related_diseases","foldchangeMaxAbs","PTMsite", "PTMtype"]


# build new csv with new column structure
def new_col_struc(old,new):
    
    with open(old,"r") as myvarfile:
        myvarread = csv.reader(myvarfile, delimiter=';') 
        with open(new,"w") as myvarfile_pred: 
            mywriter = csv.writer(myvarfile_pred, delimiter=';') 
            for myrow in myvarread:
                if myrow[0]=="Family": #first row, new names
                    mywriter.writerow(columNamesNEW)
                    continue            
                else:
                    if myrow[20]!='?' and  myrow[22]!='?':# there are impact predictors
                        #compute impact value (0,1,2)
#                         print(myrow[3])
                        impact=0#default not impact
                        predDicAux=fuse_preds(myrow[20],myrow[19],myrow[22],myrow[21])#siftScore,siftPred,ppScore,ppPred
                        #now filter the transID of the ensID of the entry name
            
            
                        predDic=clean_trans(predDicAux, myrow[2])#prediction Dic and short name
#                         print(predDic)

                        impact=compute_impact(predDic)# change to 1,2
#                         print(impact)
                    else:# no impact information
                        impact=None

                    mynewrow=write_newvar_line(myrow,impact)
                    mywriter.writerow(mynewrow)
    myvarfile.close()
    return None
                    


new_col_struc(os.path.join(resultspath,"remaining3/PRED/variants_A_remaining_GNOMAD_DISGENET_PRED.csv"),os.path.join(resultspath,"remaining3/PTM_IMPACT/vars_A_remaining_impact.csv"))

{'ENST00000296271': ['deleterious', '0', 'probably_damaging', '1']}
{'ENST00000296271': ['deleterious', '0', 'probably_damaging', '0.996']}
{'ENST00000296271': ['deleterious', '0.03', 'probably_damaging', '1']}
{'ENST00000296271': ['tolerated', '0.12', 'benign', '0']}
{'ENST00000296271': ['deleterious', '0.02', 'probably_damaging', '0.992']}
{'ENST00000296271': ['deleterious', '0.05', 'possibly_damaging', '0.777']}
{'ENST00000296271': ['deleterious', '0', 'probably_damaging', '1']}
{'ENST00000296271': ['deleterious', '0.01', 'probably_damaging', '1']}
{'ENST00000296271': ['tolerated', '0.29', 'benign', '0.003']}
{'ENST00000296271': ['deleterious', '0', 'benign', '0.045']}
{'ENST00000296271': ['deleterious', '0', 'probably_damaging', '0.909']}
{'ENST00000296271': ['tolerated', '0.06', 'probably_damaging', '0.998']}
{'ENST00000296271': ['tolerated', '0.06', 'probably_damaging', '0.998']}
{'ENST00000296271': ['tolerated', '0.06', 'benign', '0.01']}
{'ENST00000296271': ['tolerated', '0.23'

{'ENST00000296271': ['deleterious', '0', 'benign', '0.072']}
{'ENST00000296271': ['tolerated', '0.08', 'probably_damaging', '1']}
{'ENST00000296271': ['deleterious', '0', 'probably_damaging', '1']}
{'ENST00000296271': ['deleterious', '0', 'probably_damaging', '1']}
{'ENST00000296271': ['deleterious', '0', 'probably_damaging', '0.962']}
{'ENST00000296271': ['deleterious', '0.01', 'probably_damaging', '1']}
{'ENST00000296271': ['deleterious', '0.01', 'probably_damaging', '1']}
{'ENST00000296271': ['tolerated', '0.06', 'benign', '0.222']}
{'ENST00000296271': ['tolerated', '0.06', 'benign', '0.222']}
{'ENST00000296271': ['deleterious', '0.02', 'benign', '0.412']}
{'ENST00000296271': ['deleterious', '0.01', 'benign', '0.119']}
{'ENST00000296271': ['deleterious', '0.05', 'benign', '0.444']}
{'ENST00000296271': ['deleterious', '0', 'probably_damaging', '0.962']}
{'ENST00000296271': ['deleterious', '0.03', 'probably_damaging', '0.991']}
{'ENST00000296271': ['deleterious', '0.01', 'possibly_dam

{'ENST00000372071': ['tolerated_low_confidence', '0.07', 'benign', '0.372']}
{'ENST00000372071': ['tolerated_low_confidence', '0.21', 'benign', '0.003']}
{'ENST00000372071': ['tolerated_low_confidence', '0.68', 'benign', '0']}
{'ENST00000372071': ['tolerated_low_confidence', '0.21', 'benign', '0.053']}
{'ENST00000372071': ['deleterious_low_confidence', '0', 'benign', '0.015']}
{'ENST00000372071': ['tolerated', '0.32', 'benign', '0.059']}
{'ENST00000372071': ['tolerated', '0.83', 'benign', '0']}
{'ENST00000372071': ['deleterious', '0.03', 'benign', '0.018']}
{'ENST00000372071': ['tolerated', '0.26', 'benign', '0']}
{'ENST00000372071': ['tolerated', '0.25', 'benign', '0.003']}
{'ENST00000372071': ['tolerated', '0.25', 'benign', '0.003']}
{'ENST00000372071': ['tolerated', '0.13', 'benign', '0.009']}
{'ENST00000372071': ['tolerated', '0.25', 'benign', '0.031']}
{'ENST00000372071': ['tolerated', '0.17', 'benign', '0.015']}
{'ENST00000372071': ['tolerated', '0.11', 'benign', '0.025']}
{'ENST

{'ENST00000372071': ['deleterious', '0', 'possibly_damaging', '0.81']}
{'ENST00000372071': ['deleterious', '0', 'possibly_damaging', '0.81']}
{'ENST00000372071': ['deleterious', '0', 'possibly_damaging', '0.81']}
{'ENST00000372071': ['deleterious', '0', 'probably_damaging', '0.999']}
{'ENST00000372071': ['deleterious', '0', 'probably_damaging', '0.999']}
{'ENST00000372071': ['deleterious', '0', 'probably_damaging', '0.994']}
{'ENST00000372071': ['deleterious', '0.01', 'probably_damaging', '0.945']}
{'ENST00000372071': ['deleterious', '0', 'probably_damaging', '0.999']}
{'ENST00000372071': ['deleterious', '0.01', 'probably_damaging', '0.964']}
{'ENST00000372071': ['deleterious', '0', 'probably_damaging', '0.997']}
{'ENST00000372071': ['deleterious', '0', 'probably_damaging', '1']}
{'ENST00000372071': ['deleterious', '0', 'probably_damaging', '0.999']}
{'ENST00000372071': ['deleterious', '0', 'probably_damaging', '0.998']}
{'ENST00000372071': ['deleterious', '0', 'probably_damaging', '0.

{'ENST00000372071': ['deleterious', '0.03', 'possibly_damaging', '0.814']}
{'ENST00000372071': ['deleterious', '0', 'probably_damaging', '0.999']}
{'ENST00000372071': ['deleterious', '0', 'probably_damaging', '1']}
{'ENST00000372071': ['tolerated', '0.12', 'benign', '0.089']}
{'ENST00000372071': ['deleterious', '0', 'probably_damaging', '0.999']}
{'ENST00000372071': ['tolerated', '0.1', 'probably_damaging', '0.986']}
{'ENST00000372071': ['tolerated', '0.1', 'probably_damaging', '0.986']}
{'ENST00000372071': ['tolerated', '0.1', 'probably_damaging', '0.986']}
{'ENST00000372071': ['tolerated', '0.05', 'probably_damaging', '0.977']}
{'ENST00000372071': ['tolerated', '0.07', 'possibly_damaging', '0.69']}
{'ENST00000372071': ['tolerated', '0.46', 'benign', '0.05']}
{'ENST00000372071': ['deleterious', '0', 'probably_damaging', '0.943']}
{'ENST00000372071': ['deleterious', '0', 'probably_damaging', '0.999']}
{'ENST00000372071': ['deleterious', '0', 'probably_damaging', '0.999']}
{'ENST0000037

...

{'ENST00000369935': ['tolerated', '0.22', 'benign', '0.162']}
{'ENST00000369935': ['tolerated', '0.93', 'benign', '0.006']}
{'ENST00000369935': ['tolerated', '0.07', 'benign', '0.081']}
{'ENST00000369935': ['tolerated', '0.06', 'possibly_damaging', '0.524']}
{'ENST00000369935': ['tolerated', '0.4', 'benign', '0.014']}
{'ENST00000369935': ['tolerated', '0.47', 'benign', '0.066']}
{'ENST00000369935': ['tolerated', '0.79', 'benign', '0.006']}
{'ENST00000369935': ['tolerated', '1', 'benign', '0.003']}
{'ENST00000369935': ['tolerated', '0.86', 'benign', '0.054']}
{'ENST00000369935': ['tolerated', '0.1', 'benign', '0.403']}
{'ENST00000369935': ['tolerated', '0.3', 'probably_damaging', '0.999']}
{'ENST00000369935': ['deleterious', '0', 'probably_damaging', '0.967']}
{'ENST00000369935': ['deleterious', '0.01', 'probably_damaging', '0.982']}
{'ENST00000369935': ['deleterious', '0.03', 'benign', '0.179']}
{'ENST00000369935': ['deleterious', '0.01', 'probably_damaging', '0.992']}
{'ENST0000036993

{'ENST00000369951': ['deleterious', '0.01', 'benign', '0.012']}
{'ENST00000369951': ['deleterious', '0.01', 'benign', '0.012']}
{'ENST00000369951': ['tolerated', '1', 'benign', '0.005']}
{'ENST00000369951': ['tolerated', '0.32', 'benign', '0.007']}
{'ENST00000369951': ['tolerated', '0.32', 'benign', '0.007']}
{'ENST00000369951': ['tolerated', '1', 'benign', '0.001']}
{'ENST00000369951': ['tolerated', '0.26', 'benign', '0.013']}
{'ENST00000369951': ['tolerated', '0.26', 'benign', '0.013']}
{'ENST00000369951': ['tolerated', '1', 'benign', '0.013']}
{'ENST00000369951': ['deleterious', '0', 'probably_damaging', '0.989']}
{'ENST00000369951': ['deleterious', '0', 'probably_damaging', '0.989']}
{'ENST00000369951': ['tolerated', '0.05', 'possibly_damaging', '0.72']}
{'ENST00000369951': ['deleterious', '0', 'probably_damaging', '0.946']}
{'ENST00000369951': ['tolerated', '0.43', 'benign', '0.073']}
{'ENST00000369951': ['tolerated', '0.43', 'benign', '0.073']}
{'ENST00000369951': ['deleterious',

{'ENST00000328890': ['tolerated', '1', 'benign', '0.003']}
{'ENST00000328890': ['deleterious', '0.04', 'benign', '0.295']}
{'ENST00000328890': ['deleterious', '0.02', 'probably_damaging', '0.957']}
{'ENST00000328890': ['deleterious', '0.02', 'probably_damaging', '0.957']}
{'ENST00000328890': ['tolerated', '1', 'benign', '0']}
{'ENST00000328890': ['tolerated', '0.31', 'benign', '0.072']}
{'ENST00000328890': ['tolerated', '0.7', 'benign', '0']}
{'ENST00000328890': ['tolerated', '0.07', 'benign', '0.003']}
{'ENST00000328890': ['tolerated', '0.07', 'benign', '0.003']}
{'ENST00000328890': ['deleterious', '0.02', 'benign', '0.299']}
{'ENST00000328890': ['deleterious', '0.01', 'benign', '0.01']}
{'ENST00000328890': ['deleterious', '0.01', 'benign', '0.01']}
{'ENST00000328890': ['deleterious', '0.02', 'benign', '0.048']}
{'ENST00000328890': ['deleterious', '0', 'probably_damaging', '0.999']}
{'ENST00000328890': ['deleterious', '0', 'probably_damaging', '0.998']}
{'ENST00000328890': ['deleterio

{'ENST00000330803': ['tolerated', '0.86', 'benign', '0.003']}
{'ENST00000330803': ['deleterious', '0', 'probably_damaging', '0.993']}
{'ENST00000330803': ['deleterious', '0', 'probably_damaging', '0.993']}
{'ENST00000330803': ['deleterious', '0', 'probably_damaging', '1']}
{'ENST00000330803': ['tolerated', '0.17', 'benign', '0.021']}
{'ENST00000330803': ['deleterious', '0', 'probably_damaging', '0.995']}
{'ENST00000330803': ['deleterious', '0.02', 'benign', '0.167']}
{'ENST00000330803': ['deleterious', '0.02', 'benign', '0.167']}
{'ENST00000330803': ['tolerated', '0.59', 'benign', '0.122']}
{'ENST00000330803': ['deleterious', '0.03', 'possibly_damaging', '0.506']}
{'ENST00000330803': ['deleterious', '0.02', 'possibly_damaging', '0.885']}
{'ENST00000330803': ['deleterious', '0.02', 'possibly_damaging', '0.885']}
{'ENST00000330803': ['tolerated', '0.09', 'possibly_damaging', '0.766']}
{'ENST00000330803': ['tolerated', '0.94', 'benign', '0.014']}
{'ENST00000330803': ['deleterious', '0.01'

{'ENST00000330803': ['deleterious', '0', 'probably_damaging', '0.922']}
{'ENST00000330803': ['deleterious', '0', 'probably_damaging', '0.922']}
{'ENST00000330803': ['tolerated', '0.21', 'benign', '0.138']}
{'ENST00000330803': ['deleterious', '0.03', 'benign', '0.011']}
{'ENST00000330803': ['deleterious', '0.01', 'benign', '0.045']}
{'ENST00000330803': ['deleterious', '0.02', 'benign', '0.094']}
{'ENST00000330803': ['deleterious', '0', 'probably_damaging', '0.999']}
{'ENST00000330803': ['deleterious', '0.02', 'benign', '0.013']}
{'ENST00000330803': ['tolerated', '0.17', 'possibly_damaging', '0.536']}
{'ENST00000330803': ['tolerated', '0.06', 'benign', '0']}
{'ENST00000330803': ['tolerated', '0.06', 'benign', '0']}
{'ENST00000330803': ['tolerated', '0.05', 'benign', '0.036']}
{'ENST00000330803': ['tolerated', '0.64', 'benign', '0']}
{'ENST00000330803': ['deleterious', '0.01', 'benign', '0.017']}
{'ENST00000330803': ['deleterious', '0.01', 'benign', '0.017']}
{'ENST00000330803': ['deleter

#### 1.6.2 GPCR position and Role corrections
Now, the last file "PTM_IMPACT/vars_A_unf_impact_ptm.csv" has few details to correct. The comma separation of the frequency column should be dots (we will see how we deal with this), the GPCR position should have the format helixXposition and we will add the role if the variant is in some of the know positions. We will also add an index (to be used later in the data frames).

We will use the 'structural' GPCR numeration, the one developed by GPCRbd (the notation with the 'x' not with the dot. It igones the position number when there is a deletion or adds an extra number when there is an insertion (something that the notation with dots doesn't do). Note that when there is no GPCRnum is because we are in a flexible loop or one of the terminals. 

In [18]:
################################THIS IS ONLY VALID FOR CLASS A ############################
def set_role(myrow):
    '''Set which is the function of the AA: BS, IBS, CM depending on the variant position '''
    position=myrow[9]
    bs=['23x49'] # selected position from BS
    ibs=['12x48','4x39','6x29'] # IBS (3)
    cm_il=['3x50'] # CM-IL (also of IBS)
    cm_rts=[]
    cm_
    if position in bs:
        function= 'LB_structure'
    elif position in ibs: 
        if position=='12x48':
            function="GProtein/ArrestinInteraction (chain B)"
        else:
            function="GProtein/ArrestinInteraction"
    elif position in cm_il:
        function='IonicLock'
    else:
        function='?'
    return function
####################################################################################

def new_col_struc_bis(old,new): 
    '''Change the gpcr numeration, add an index and set the role
    of each variant, define the role'''
    #bs to review* only aminergic
    bs=['7x37', '6x59', '5x461', '5x37', '23x49', '7x41', '3x29', '2x63', '2x53', '3x35', '6x44', '6x51', '7x35', '3x28', '7x38', '7x31', '6x48', '3x33', '6x52', '6x58', '7x39', '6x56', '3x40', '6x55', '45x50', '5x39', '45x51', '4x57', '5x36', '7x34', '4x56', '5x40', '7x30', '23x50', '2x64', '2x56', '3x37', '5x47', '6x54', '2x60', '2x59', '3x32', '3x36', '5x44', '45x52', '7x42', '5x43', '3x25']
    ibs=['1x59', '1x60', '12x48', '12x49', '12x50', '12x51', '2x34', '2x35', '2x36', '2x37', '2x38', '2x39', '2x40', '2x43', '3x49', '3x50', '3x53', '3x54', '3x55', '3x56', '34x50', '34x51', '34x52', '34x53', '34x54', '34x55', '34x56', '34x57', '4x39', '4x40', '5x61', '5x64', '5x65', '5x66', '5x67', '5x68', '5x69', '5x71', '5x72', '5x73', '5x74', '5x75', '6x24', '6x25', '6x26', '6x28', '6x29', '6x30', '6x32', '6x33', '6x36', '6x37', '6x38', '6x40', '7x53', '7x55', '7x56', '8x47', '8x48', '8x49']
    cm_naBS=['2x50', '3x39', '7x45', '7x46']
    cm_ionicLock=['3x50', '6x30']
    cm_rotTogg=['6x48']
    count=0
    with open(old,"r") as myvarfile:
        myvarread = csv.reader(myvarfile, delimiter=';') 
        with open(new,"w") as myvarfile_pred: 
            mywriter = csv.writer(myvarfile_pred, delimiter=';') 
            for myrow in myvarread:
                if myrow[0]=="Family": #first row
                    mywriter.writerow(myrow)
                    continue            
                else:
                    mynewrow=myrow
                    #frequency commas correction, necessary?
#                     myNewRow[15]=myrow[15].replace(',','.')#freq now dot delimiter
#                     print(myNewRow[15])

                    #gnum
                    if myrow[8]!='' and myrow[8]!='?':#there is gpcrnum
                        
                        gnum=myrow[8] #a.bc x cd
                        # Split: a.bc, cd
                        gnum1=gnum.split('x')[0]# a.bc,cd
#                         print(gnum1)
    #                     gnum2=gnum1.split('.')[1] #bc, we don't use this notation
                        helix=gnum1.split('.')[0] #a                      
                        position=gnum.split('x')[1]#cd, we use this, the structural notation
                        gpcrnum=helix+'x'+position#simple gpcr numeration
                        mynewrow[8]=gpcrnum
                    
                    #role
                        role=''
                        if gpcrnum in bs:
                            role='Binding site'
                        if gpcrnum in ibs:
                            role='Intracellular binding site'
                        if gpcrnum in cm_naBS:
                            role='CM: Na binding site'
                        if gpcrnum in cm_ionicLock:
                            role='CM: ionic lock'
                        if gpcrnum in cm_rotTogg:
                            role='CM: rotamer toggle switch'
                        if role!='':
                            mynewrow[17]=role
                    
                    #index
                    mynewrow.insert(0, count)
                    mywriter.writerow(mynewrow)  
                    count+=1
        myvarfile_pred.close()     
    myvarfile.close()
    return (print('File OK'))

new_col_struc_bis(os.path.join(resultspath,"remaining3/PTM_IMPACT/vars_A_remaining_impact.csv"),os.path.join(resultspath,"remaining3/PTM_IMPACT/vars_A_remaining_impact_role.csv"))

File OK


In [19]:
def add_disbool(old, new):
    ''' Add columb with T/F values to know if the variant is related or not with a disease'''
    columNamesNEW_with_index=["Number","Family","Name","Short","Uniprot ID","Uniprot entry","Struc","gene","varID",
               "GPCRdb","SequenceNumber","Segment","GPCRdbWT","NMaa","MutationType",
               "Allele Count", "Allele Frequency","Allele Number","Role","Impact Prediction",
                "diseaseId","score","diseaseName","related_diseases",'foldchangeMaxAbs',"PTM","PTMtype","disease"]
                             
    with open(os.path.join(resultspath,old),"r") as myvarfile:
        myvarread = csv.reader(myvarfile, delimiter=';') 
        with open(os.path.join(resultspath,new),"w") as myvarfile_pred: 
            mywriter = csv.writer(myvarfile_pred, delimiter=';') 
            for myrow in myvarread:
                if myrow[0]=="Number": #first row, new names
                    mywriter.writerow(columNamesNEW_with_index)
                    continue            
                else:
                    mynewrow=myrow
                    if myrow[20]!='?':#disease related
                        mynewrow.append('Yes')
                    else: 
                        mynewrow.append('No')
                    mywriter.writerow(mynewrow)
    return None
add_disbool(os.path.join(resultspath,"remaining3/PTM_IMPACT/vars_A_remaining_impact_role.csv"),os.path.join(resultspath,"remaining3/PTM_IMPACT/vars_A_remaining_impact_role1.csv"))


Then, manually edited, foldchange out, ptm to the end, header moved to put 'Number' first.